In [1]:
from __future__ import print_function
import os
data_path = [ './']

from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

In [2]:
# read file
import pandas as pd
import numpy as np

filepath = os.sep.join(data_path + ['pd_speech_features.csv'])
data = pd.read_csv(filepath, sep=',')

In [3]:
data

,id,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,...,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
0,0,1,0.85247,0.71826,0.57227,240,239,0.008064,0.000087,0.00218,...,1.5620,2.6445,3.8686,4.2105,5.1221,4.4625,2.6202,3.0004,18.9405,1
1,0,1,0.76686,0.69481,0.53966,234,233,0.008258,0.000073,0.00195,...,1.5589,3.6107,23.5155,14.1962,11.0261,9.5082,6.5245,6.3431,45.1780,1
2,0,1,0.85083,0.67604,0.58982,232,231,0.008340,0.000060,0.00176,...,1.5643,2.3308,9.4959,10.7458,11.0177,4.8066,2.9199,3.1495,4.7666,1
3,1,0,0.41121,0.79672,0.59257,178,177,0.010858,0.000183,0.00419,...,3.7805,3.5664,5.2558,14.0403,4.2235,4.6857,4.8460,6.2650,4.0603,1
4,1,0,0.32790,0.79782,0.53028,236,235,0.008162,0.002669,0.00535,...,6.1727,5.8416,6.0805,5.7621,7.7817,11.6891,8.2103,5.0559,6.1164,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,250,0,0.80903,0.56355,0.28385,417,416,0.004627,0.000052,0.00064,...,3.0706,3.0190,3.1212,2.4921,3.5844,3.5400,3.3805,3.2003,6.8671,0
752,250,0,0.16084,0.56499,0.59194,415,413,0.004550,0.000220,0.00143,...,1.9704,1.7451,1.8277,2.4976,5.2981,4.2616,6.3042,10.9058,28.4170,0
753,251,0,0.88389,0.72335,0.46815,381,380,0.005069,0.000103,0.00076,...,51.5607,44.4641,26.1586,6.3076,2.8601,2.5361,3.5377,3.3545,5.0424,0
754,251,0,0.83782,0.74890,0.49823,340,339,0.005679,0.000055,0.00092,...,19.1607,12.8312,8.9434,2.2044,1.9496,1.9664,2.6801,2.8332,3.7131,0


In [4]:
# drop 'id' column
data = data.drop('id', axis=1)
data

,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,locAbsJitter,...,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
0,1,0.85247,0.71826,0.57227,240,239,0.008064,0.000087,0.00218,0.000018,...,1.5620,2.6445,3.8686,4.2105,5.1221,4.4625,2.6202,3.0004,18.9405,1
1,1,0.76686,0.69481,0.53966,234,233,0.008258,0.000073,0.00195,0.000016,...,1.5589,3.6107,23.5155,14.1962,11.0261,9.5082,6.5245,6.3431,45.1780,1
2,1,0.85083,0.67604,0.58982,232,231,0.008340,0.000060,0.00176,0.000015,...,1.5643,2.3308,9.4959,10.7458,11.0177,4.8066,2.9199,3.1495,4.7666,1
3,0,0.41121,0.79672,0.59257,178,177,0.010858,0.000183,0.00419,0.000046,...,3.7805,3.5664,5.2558,14.0403,4.2235,4.6857,4.8460,6.2650,4.0603,1
4,0,0.32790,0.79782,0.53028,236,235,0.008162,0.002669,0.00535,0.000044,...,6.1727,5.8416,6.0805,5.7621,7.7817,11.6891,8.2103,5.0559,6.1164,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,0,0.80903,0.56355,0.28385,417,416,0.004627,0.000052,0.00064,0.000003,...,3.0706,3.0190,3.1212,2.4921,3.5844,3.5400,3.3805,3.2003,6.8671,0
752,0,0.16084,0.56499,0.59194,415,413,0.004550,0.000220,0.00143,0.000006,...,1.9704,1.7451,1.8277,2.4976,5.2981,4.2616,6.3042,10.9058,28.4170,0
753,0,0.88389,0.72335,0.46815,381,380,0.005069,0.000103,0.00076,0.000004,...,51.5607,44.4641,26.1586,6.3076,2.8601,2.5361,3.5377,3.3545,5.0424,0
754,0,0.83782,0.74890,0.49823,340,339,0.005679,0.000055,0.00092,0.000005,...,19.1607,12.8312,8.9434,2.2044,1.9496,1.9664,2.6801,2.8332,3.7131,0


In [5]:
y = data['class']
fields = list(data.columns[:-1])
correlations = data[fields].corrwith(y)
correlations.sort_values(inplace=True)
correlations

tqwt_stdValue_dec_12      -0.393029
tqwt_maxValue_dec_12      -0.391288
tqwt_stdValue_dec_11      -0.391081
tqwt_entropy_log_dec_12   -0.381003
tqwt_maxValue_dec_11      -0.378485
                             ...   
std_9th_delta_delta        0.364333
tqwt_minValue_dec_13       0.365264
tqwt_minValue_dec_11       0.366552
tqwt_minValue_dec_12       0.395740
mean_MFCC_2nd_coef         0.398695
Length: 753, dtype: float64

In [7]:
# feature columns
feature_cols = [x for x in data.columns if x not in 'class']

In [8]:
# Train test split with StratifiedShuffleSplit
strat_shuff_split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

# Get the index values from the generator
train_idx, test_idx = next(strat_shuff_split.split(data[feature_cols], data['class']))

# Create the data sets
X_train = data.loc[train_idx, feature_cols]
y_train = data.loc[train_idx, 'class']

X_test = data.loc[test_idx, feature_cols]
y_test = data.loc[test_idx, 'class']

In [9]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((529, 753), (227, 753), (529,), (227,))

In [10]:
# Label proportion
print(y_train.value_counts(normalize=True).sort_index())
print(y_test.value_counts(normalize=True).sort_index())

0    0.253308
1    0.746692
Name: class, dtype: float64
0    0.255507
1    0.744493
Name: class, dtype: float64


In [11]:
# MinMaxScaler
mmsc = MinMaxScaler()

X_minmax_train = pd.DataFrame(mmsc.fit_transform(X_train), columns=feature_cols)
X_minmax_test = pd.DataFrame(mmsc.transform(X_test), columns=feature_cols)
X_minmax_train

,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,locAbsJitter,...,tqwt_kurtosisValue_dec_27,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36
0,1.0,0.717368,0.928834,0.685291,0.227778,0.234160,0.569466,0.035103,0.196078,0.171286,...,0.001253,0.001096,0.017467,0.017630,0.016304,0.024077,0.040765,0.040284,0.058379,0.021953
1,1.0,0.900625,0.394121,0.649490,0.268056,0.274105,0.481645,0.030425,0.092593,0.073551,...,0.000851,0.004109,0.334035,0.545107,0.635907,0.771423,0.806359,0.696448,0.725671,0.762038
2,0.0,0.900197,0.379072,0.153789,0.436111,0.440771,0.256773,0.003687,0.014524,0.009007,...,0.381417,0.275101,0.324983,0.261984,0.087155,0.047391,0.043529,0.028637,0.032050,0.003375
3,1.0,0.905174,0.301318,0.560331,0.243056,0.249311,0.532487,0.015697,0.032680,0.030079,...,0.001038,0.001421,0.005503,0.010652,0.013160,0.020005,0.039187,0.026685,0.017500,0.008683
4,0.0,0.913937,0.318373,0.280273,0.347222,0.352617,0.355249,0.011089,0.026870,0.019054,...,0.024832,0.584298,0.584802,0.616315,0.723429,0.934462,0.901537,0.824702,0.885593,0.769444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524,1.0,0.359192,0.900676,0.683034,0.208333,0.214876,0.615728,0.066226,0.281046,0.257684,...,0.001116,0.000312,0.013187,0.183655,0.173538,0.153170,0.214697,0.691071,0.759223,0.417778
525,0.0,0.928762,0.006923,0.440805,0.565278,0.568871,0.160398,0.012961,0.013072,0.006153,...,0.739682,0.721640,0.771483,0.858765,0.857194,0.806341,0.765640,0.653818,0.388005,0.006631
526,1.0,0.903292,0.524982,0.622070,0.237500,0.243802,0.541684,0.028198,0.064633,0.056741,...,0.001386,0.000945,0.044032,0.017818,0.013503,0.016896,0.019738,0.076578,0.082092,0.031648
527,0.0,0.912990,0.645910,0.461898,0.438889,0.443526,0.254432,0.018203,0.038126,0.021400,...,0.278231,0.239727,0.317335,0.314255,0.095970,0.075010,0.084877,0.502512,0.807987,0.764781


In [12]:
# StandardScaler
stdsc = StandardScaler()

X_standard_train = pd.DataFrame(stdsc.fit_transform(X_train), columns=feature_cols)
X_standard_test = pd.DataFrame(stdsc.transform(X_test), columns=feature_cols)
X_standard_train

,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,locAbsJitter,...,tqwt_kurtosisValue_dec_27,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36
0,0.961060,-0.523756,1.851295,1.005378,-0.829721,-0.824966,0.846335,-0.343173,1.094912,1.055203,...,-0.468858,-0.604355,-0.582675,-0.598357,-0.536424,-0.490521,-0.499304,-0.689215,-0.702862,-0.790164
1,0.961060,0.447063,-0.424657,0.828423,-0.532341,-0.528118,0.385059,-0.365631,0.111419,0.069853,...,-0.471066,-0.590013,0.963551,1.857822,2.491616,2.891754,2.724397,1.761654,1.576951,1.749492
2,-1.040518,0.444800,-0.488712,-1.621665,0.708451,0.710453,-0.796066,-0.493999,-0.630515,-0.580872,...,1.619098,0.699845,0.919336,0.539470,-0.190170,-0.385008,-0.487669,-0.732718,-0.792816,-0.853915
3,0.961060,0.471162,-0.819664,0.387739,-0.716922,-0.712368,0.652104,-0.436337,-0.457972,-0.368431,...,-0.470042,-0.602806,-0.641111,-0.630850,-0.551787,-0.508946,-0.505949,-0.740008,-0.842526,-0.835700
4,-1.040518,0.517587,-0.747068,-0.996493,0.052164,0.055341,-0.278828,-0.458462,-0.513186,-0.479578,...,-0.339358,2.171545,2.188379,2.189402,2.919339,3.629624,3.125163,2.240701,2.123328,1.774908
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524,0.961060,-2.421234,1.731441,0.994221,-0.973283,-0.968271,1.089320,-0.193756,1.902412,1.926252,...,-0.469611,-0.608085,-0.603580,0.174731,0.231989,0.093718,0.233072,1.741571,1.691581,0.568141
525,-1.040518,0.596124,-2.072725,-0.203036,1.662117,1.662412,-1.302270,-0.449474,-0.644318,-0.609644,...,3.586780,2.825257,3.100195,3.318361,3.573057,3.049783,2.552939,1.602425,0.423310,-0.842740
526,0.961060,0.461192,0.132339,0.692899,-0.757939,-0.753313,0.700409,-0.376321,-0.154297,-0.099627,...,-0.468132,-0.605070,-0.452922,-0.597485,-0.550113,-0.523018,-0.587846,-0.553654,-0.621847,-0.756894
527,-1.040518,0.512571,0.647058,-0.098782,0.728959,0.730925,-0.808360,-0.424307,-0.406209,-0.455930,...,1.052373,0.531472,0.881979,0.782869,-0.147093,-0.260011,-0.313563,1.037275,1.858186,1.758907


In [13]:
# Specificity score function
def specificity_score(y_true, y_pred):
    cf = confusion_matrix(y_true, y_pred)
    return cf[0,0]/(cf[0,0]+cf[0,1])

# Error measure function
def measure_error(y_true, y_pred, label):
    return pd.Series({'accuracy':accuracy_score(y_true, y_pred),
                      'precision': precision_score(y_true, y_pred),
                      'recall': recall_score(y_true, y_pred),
                      'specificity': specificity_score(y_true, y_pred),
                      'balanced accuracy': balanced_accuracy_score(y_true, y_pred),
                      'f1': f1_score(y_true, y_pred)},
                      name=label)

In [ ]:
# Linear SVM

In [14]:
# Linear SVM with no-scaling
LSVC = LinearSVC()
LSVC.fit(X_train, y_train)

y_train_pred = LSVC.predict(X_train)
y_test_pred = LSVC.predict(X_test)

print(confusion_matrix(y_true=y_test, y_pred=y_test_pred))

train_test_error = pd.concat([measure_error(y_train, y_train_pred, 'train'), 
                              measure_error(y_test, y_test_pred, 'test')], axis=1)
train_test_error

[[ 56   2]
 [165   4]]


/opt/homebrew/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


,train,test
accuracy,0.262760,0.264317
precision,1.000000,0.666667
recall,0.012658,0.023669
specificity,1.000000,0.965517
balanced accuracy,0.506329,0.494593
f1,0.025000,0.045714


In [15]:
# Linear SVM with MinMaxScaler
LSVC = LinearSVC(max_iter=10000)
LSVC.fit(X_minmax_train, y_train)

y_minmax_train_pred = LSVC.predict(X_minmax_train)
y_minmax_test_pred = LSVC.predict(X_minmax_test)

print(confusion_matrix(y_true=y_test, y_pred=y_minmax_test_pred))

minmax_train_test_error = pd.concat([measure_error(y_train, y_minmax_train_pred, 'minmax_train'), 
                                     measure_error(y_test, y_minmax_test_pred, 'minmax_test')], axis=1)
minmax_train_test_error


[[ 38  20]
 [  8 161]]


,minmax_train,minmax_test
accuracy,1.0,0.876652
precision,1.0,0.889503
recall,1.0,0.952663
specificity,1.0,0.655172
balanced accuracy,1.0,0.803918
f1,1.0,0.920000


In [16]:
# Linear SVM with StandardScaler
LSVC = LinearSVC(max_iter=10000)
LSVC.fit(X_standard_train, y_train)

y_standard_train_pred = LSVC.predict(X_standard_train)
y_standard_test_pred = LSVC.predict(X_standard_test)

print(confusion_matrix(y_true=y_test, y_pred=y_standard_test_pred))

standard_train_test_error = pd.concat([measure_error(y_train, y_standard_train_pred, 'standard_train'), 
                                       measure_error(y_test, y_standard_test_pred, 'standard_test')], axis=1)
standard_train_test_error


[[ 38  20]
 [ 19 150]]


,standard_train,standard_test
accuracy,1.0,0.828194
precision,1.0,0.882353
recall,1.0,0.887574
specificity,1.0,0.655172
balanced accuracy,1.0,0.771373
f1,1.0,0.884956


In [ ]:
# Gaussian Kernel SVM

In [17]:
# Gaussian kernel SVM with no-scaling
SVC_Gaussian = SVC(kernel='rbf')
GS = GridSearchCV(SVC_Gaussian, 
                  param_grid={
                      'gamma':[0.0001, 0.001, 0.01, 0.1, 1, 10], 
                      'C':[0.0001, 0.001, 0.01, 0.1, 1, 10, 100]}, 
                  scoring=['accuracy', 'recall', 'balanced_accuracy'],
                  refit='balanced_accuracy')

GS.fit(X_train, y_train)

GS.best_params_, GS.best_score_

({'C': 0.0001, 'gamma': 0.0001}, 0.5)

In [18]:
# Best model of Gaussian Kernel SVM with no-scaling
SVC_Gaussian = SVC(kernel='rbf', gamma=0.0001, C=0.0001)
SVC_Gaussian.fit(X_train, y_train)

y_train_pred = SVC_Gaussian.predict(X_train)
y_test_pred = SVC_Gaussian.predict(X_test)

print(confusion_matrix(y_true=y_test, y_pred=y_test_pred))

train_test_error = pd.concat([measure_error(y_train, y_train_pred, 'train'), 
                              measure_error(y_test, y_test_pred, 'test')], axis=1)
train_test_error

[[  0  58]
 [  0 169]]


,train,test
accuracy,0.746692,0.744493
precision,0.746692,0.744493
recall,1.000000,1.000000
specificity,0.000000,0.000000
balanced accuracy,0.500000,0.500000
f1,0.854978,0.853535


In [18]:
# Gaussian kernel SVM with MinMaxScaler
SVC_Gaussian = SVC(kernel='rbf')
GS = GridSearchCV(SVC_Gaussian, 
                  param_grid={
                      'gamma':[0.0001, 0.001, 0.01, 0.1, 1, 10], 
                      'C':[0.0001, 0.001, 0.01, 0.1, 1, 10, 100]}, 
                  scoring=['accuracy', 'recall', 'balanced_accuracy'],
                  refit='balanced_accuracy')

GS.fit(X_minmax_train, y_train)

GS.best_params_, GS.best_score_

({'C': 10, 'gamma': 0.1}, 0.830267950521115)

In [19]:
# Gaussian kernel SVM with StandardScaler
SVC_Gaussian = SVC(kernel='rbf')
GS = GridSearchCV(SVC_Gaussian, 
                  param_grid={
                      'gamma':[0.0001, 0.001, 0.01, 0.1, 1, 10], 
                      'C':[0.0001, 0.001, 0.01, 0.1, 1, 10, 100]}, 
                  scoring=['accuracy', 'recall', 'balanced_accuracy'],
                  refit='balanced_accuracy')

GS.fit(X_standard_train, y_train)

GS.best_params_, GS.best_score_

({'C': 10, 'gamma': 0.001}, 0.7866168992751271)

In [ ]:
# Overall Best Model

In [20]:
# Gaussian kernel SVM with MinMaxScaler, gamma=0.1, C=10
SVC_Gaussian = SVC(kernel='rbf', gamma=0.1, C=10)
SVC_Gaussian.fit(X_minmax_train, y_train)

y_minmax_train_pred = SVC_Gaussian.predict(X_minmax_train)
y_minmax_test_pred = SVC_Gaussian.predict(X_minmax_test)

print(confusion_matrix(y_true=y_test, y_pred=y_minmax_test_pred))

minmax_train_test_error = pd.concat([measure_error(y_train, y_minmax_train_pred, 'minmax_train'), 
                                     measure_error(y_test, y_minmax_test_pred, 'minmax_test')], axis=1)
minmax_train_test_error

[[ 44  14]
 [  4 165]]


,minmax_train,minmax_test
accuracy,1.0,0.920705
precision,1.0,0.921788
recall,1.0,0.976331
specificity,1.0,0.758621
balanced accuracy,1.0,0.867476
f1,1.0,0.948276
